In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [10]:
!mkdir -p /content/CTC360/{fastq,trimmed,qc,bam,variants,figures,ref}
project_dir = "/content/CTC360"
print("Project directory created.")


Project directory created.


In [11]:
!apt-get update
!apt-get install -y sra-toolkit fastqc bwa samtools fastp


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,143 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [13]:
!cd /content
!wget -c https://github.com/broadinstitute/gatk/releases/download/4.4.0.0/gatk-4.4.0.0.zip
!unzip -o gatk-4.4.0.0.zip


--2025-11-15 16:15:36--  https://github.com/broadinstitute/gatk/releases/download/4.4.0.0/gatk-4.4.0.0.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/27452807/0bd42ce9-3dc6-4814-971c-264ebaf66d3d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-15T17%3A15%3A40Z&rscd=attachment%3B+filename%3Dgatk-4.4.0.0.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-15T16%3A15%3A37Z&ske=2025-11-15T17%3A15%3A40Z&sks=b&skv=2018-11-09&sig=F9fY%2FLEaVBEUvjPzODMwCMesFcBvIMnkzOjV74dhJNs%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzIyNjkzNywibmJmIjoxNzYzMjIzMzM3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi

In [14]:
!cd /content/CTC360/ref

# Download only chromosome 12 FASTA (hg38)
!wget -c https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr12.fa.gz
!gunzip chr12.fa.gz
!mv chr12.fa hg38.fa

# Index reference
!bwa index hg38.fa
!samtools faidx hg38.fa
!gatk-4.4.0.0/gatk CreateSequenceDictionary -R hg38.fa


--2025-11-15 16:17:58--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr12.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42940449 (41M) [application/x-gzip]
Saving to: ‘chr12.fa.gz’

chr12.fa.gz         100%[===================>]  40.95M  26.8MB/s    in 1.5s    

2025-11-15 16:18:00 (26.8 MB/s) - ‘chr12.fa.gz’ saved [42940449/42940449]

[bwa_index] Pack FASTA... 1.07 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=266550618, availableWord=30755168
[BWTIncConstructFromPacked] 10 iterations done. 50732074 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 93723498 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 131930362 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 165884698 characte

In [ ]:
!cd /content/CTC360/fastq

# Download SRA files
!prefetch SRR13973704
!prefetch SRR13973878

# Convert to FASTQ
!fasterq-dump SRR13973704 -O .
!fasterq-dump SRR13973878 -O .

# Compress
!gzip SRR13973704_1.fastq SRR13973704_2.fastq
!gzip SRR13973878_1.fastq SRR13973878_2.fastq



2025-11-15T16:20:52 prefetch.2.11.3: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2025-11-15T16:20:52 prefetch.2.11.3: 1) Downloading 'SRR13973704'...
2025-11-15T16:20:52 prefetch.2.11.3: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2025-11-15T16:20:52 prefetch.2.11.3:  Downloading via HTTPS...
2025-11-15T16:21:14 prefetch.2.11.3:  HTTPS download succeed
2025-11-15T16:21:22 prefetch.2.11.3:  'SRR13973704' is valid
2025-11-15T16:21:22 prefetch.2.11.3: 1) 'SRR13973704' was downloaded successfully
2025-11-15T16:21:27 prefetch.2.11.3: 'SRR13973704' has 25 unresolved dependencies
2025-11-15T16:21:27 prefetch.2.11.3: 2) Downloading 'ncbi-acc:CM000663.1?vdb-ctx=refseq'...
2025-11-15T16:21:27 prefetch.2.11.3:  Downloading via HTTPS...
2025-11-15T16:21:28 prefetch.2.11.3:  HTTPS download succeed
2025-11-15T16:21:28 prefetch.2.11.3: 2) 'ncbi-acc:CM000663.1